In [9]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [11]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2687
Dimensão vocabulario positivo: 2326


In [12]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))

In [105]:
# Rankeando a incidencia das palavras de cada léxico em suas respectivas classes
from nltk import FreqDist

neg_text_tokenized = ' '.join(neg_data["texts"]).split()
pos_text_tokenized = ' '.join(pos_data["texts"]).split()

fd = FreqDist(word for word in neg_text_tokenized if word in vocab_neg)
s = fd.most_common()
s




[('\x93o', 14),
 ('sabia', 13),
 ('edison', 12),
 ('sabesp', 11),
 ('alves', 11),
 ('vaccari', 10),
 ('presidentes', 10),
 ('contas', 10),
 ('morto', 10),
 ('câmara', 10),
 ('coisa', 9),
 ('renan', 9),
 ('tesoureiro', 9),
 ('calheiros', 9),
 ('beneficiários', 8),
 ('criticar', 8),
 ('jânio', 7),
 ('negromonte', 7),
 ('últimos', 7),
 ('manteve', 7),
 ('gay', 7),
 ('causa', 7),
 ('obras', 7),
 ('parece', 7),
 ('conselho', 7),
 ('mário', 7),
 ('rn', 7),
 ('al', 7),
 ('paulista', 6),
 ('usou', 6),
 ('mesma', 6),
 ('tse', 6),
 ('governadora', 6),
 ('criou', 6),
 ('roseana', 6),
 ('água', 6),
 ('maranhão', 6),
 ('superior', 6),
 ('semanas', 6),
 ('valores', 6),
 ('cenário', 6),
 ('abastecimento', 6),
 ('s', 6),
 ('peemedebista', 6),
 ('respeito', 6),
 ('democracia', 6),
 ('levou', 6),
 ('reais', 5),
 ('inclui', 5),
 ('folha', 5),
 ('horas', 5),
 ('palestras', 5),
 ('decidiu', 5),
 ('pi', 5),
 ('saiba', 5),
 ('ciro', 5),
 ('sc', 5),
 ('tentou', 5),
 ('racionamento', 5),
 ('nogueira', 5),
 ('p

In [17]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(2,2), stop_words= stopwords_pt, binary= True)),
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = True))
                    ])

folds = 7;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.841584158416
Fold  1  - Acuracia:  0.80198019802
Fold  2  - Acuracia:  0.83
Fold  3  - Acuracia:  0.81
Fold  4  - Acuracia:  0.78
Fold  5  - Acuracia:  0.78
Fold  6  - Acuracia:  0.848484848485

Accuracia media:  0.813149886417
Desvio padrão:  0.0258388062486


SVM
Cross Validation:
Fold  0  - Acuracia:  0.80198019802
Fold  1  - Acuracia:  0.80198019802
Fold  2  - Acuracia:  0.76
Fold  3  - Acuracia:  0.87
Fold  4  - Acuracia:  0.77
Fold  5  - Acuracia:  0.68
Fold  6  - Acuracia:  0.777777777778

Accuracia media:  0.780248310545
Desvio padrão:  0.0528755477847

Maxent
Cross Validation:
Fold  0  - Acuracia:  0.851485148515
Fold  1  - Acuracia:  0.821782178218
Fold  2  - Acuracia:  0.82
Fold  3  - Acuracia:  0.88
Fold  4  - Acuracia:  0.79
Fold  5  - Acuracia:  0.78
Fold  6  - Acuracia:  0.808080808081

Accuracia media:  0.821621162116
Desvio padrão:  0.0321117706634


In [96]:
# Avaliando a incidência dos bigramas
cv = CountVectorizer(ngram_range=(3,3))

neg_counts = cv.fit_transform(neg_data["texts"])
neg_bigrams = cv.vocabulary_

pos_counts = cv.fit_transform(pos_data["texts"])
pos_bigrams = cv.vocabulary_

sp = np.sum(pos_counts.toarray(), axis=0)
sn = np.sum(neg_counts.toarray(), axis=0)

pos_bigrams  = {bigram: sp[index] for bigram, index in pos_bigrams.items()}
neg_bigrams  = {bigram: sn[index] for bigram, index in neg_bigrams.items()}

pos_excl_bigrams = {key:value for key, value in pos_bigrams.items() if key not in neg_bigrams}
neg_excl_bigrams = {key:value for key, value in neg_bigrams.items() if key not in pos_bigrams}


In [97]:
from pandas import Series
s = Series(neg_excl_bigrams)
s.nlargest(15)

henrique eduardo alves          10
eduardo alves pmdb               9
joão vaccari neto                8
lobão minas energia              8
ministro edison lobão            8
renan calheiros pmdb             8
edison lobão minas               7
pt joão vaccari                  7
alves pmdb rn                    6
calheiros pmdb al                6
envolvidos esquema corrupção     6
tesoureiro pt joão               6
adversária marina silva          5
ciro nogueira pp                 5
câmara henrique eduardo          5
dtype: int64

In [82]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

lexicon_bigrams = list(pos_excl_bigrams.keys()) + list(neg_excl_bigrams.keys())

features = FeatureUnion([
                    ("bigram", CountVectorizer(ngram_range=(1,1), stop_words= stopwords_pt, vocabulary  = lexicon)),
                    ("lexicon_vector", CountVectorizer(ngram_range=(2,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds=10)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=10)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.845070422535
Fold  1  - Acuracia:  0.842857142857
Fold  2  - Acuracia:  0.814285714286
Fold  3  - Acuracia:  0.885714285714
Fold  4  - Acuracia:  0.885714285714
Fold  5  - Acuracia:  0.842857142857
Fold  6  - Acuracia:  0.842857142857
Fold  7  - Acuracia:  0.857142857143
Fold  8  - Acuracia:  0.871428571429
Fold  9  - Acuracia:  0.871428571429

Accuracia media:  0.855935613682
Desvio padrão:  0.0214964304664


SVM
Cross Validation:
Fold  0  - Acuracia:  0.816901408451
Fold  1  - Acuracia:  0.857142857143
Fold  2  - Acuracia:  0.814285714286
Fold  3  - Acuracia:  0.8
Fold  4  - Acuracia:  0.814285714286
Fold  5  - Acuracia:  0.842857142857
Fold  6  - Acuracia:  0.8
Fold  7  - Acuracia:  0.728571428571
Fold  8  - Acuracia:  0.757142857143
Fold  9  - Acuracia:  0.757142857143

Accuracia media:  0.798832997988
Desvio padrão:  0.0381546235159
